# Load Himawari-8 AHI data and generate true color RGB

## Loading the data

In [1]:
from satpy import Scene, find_files_and_readers
from satpy.resample import get_area_def
from datetime import datetime

<p>We start by locating and specifying the AHI data to read:</p>

In [2]:
files = find_files_and_readers(start_time=datetime(2015, 2, 7, 3, 0),
                               end_time=datetime(2015, 2, 7, 3, 10),
                               base_dir="/data/satellite/Himawari-8/HSD",
                               reader='ahi_hsd')

This example is loading the Himawari data in the Himawari Standard Data (HSD) format. But you can load HRIT formatted data as well with the same code, except specifying the adequate reader (`hrit_ahi`) in the argument list above.

<p>We create the scene object</p>

In [3]:
scn = Scene(sensor='ahi', filenames=files)

Here we load the `true_color` composite

In [4]:
rgbname = 'true_color'

In [5]:
scn.load([rgbname])

/home/a001673/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Generate a full disk image

Beware that this step might need more than 10 GB memory available on the processing machine (depending on the number of cpu cores)

In [6]:
new_scn = scn.resample(scn.min_area(), resampler='native')

The above line lowers the resolution to the coarser channel. If you want instead to generate a full resolution image, use instead: `scn.max_area()` in the resample call. This will however need quite a lot of resources.

In [7]:
new_scn.save_dataset(rgbname, filename='himawari_ahi_truecolor_{datetime}.png'.format(datetime=scn.start_time.strftime('%Y%m%d%H%M')))


<img src='https://media.githubusercontent.com/media/pytroll/pytroll-examples/321c1cc5799832b8cc5b9e0994a0c461cf11213d/satpy/himawari_ahi_truecolor_201502070300_thumb.png'>

## Generate the composite on a Region of Interest
Now let us define a map-projection and a sub area, and project the data on this area. We use Pyresample to define the area. This definition can also be put in the area.yaml configuration file

In [8]:
from pyresample.utils import get_area_def
area_id = 'japan'
x_size = 2407
y_size = 1655
area_extent = (-1203877.326193321, -754860.2505908236, 1203877.3261933187, 900373.6358697552)
projection = '+proj=laea +lat_0=37.5 +lon_0=137.5 +ellps=WGS84'
description = "Japan"
proj_id = 'laea_137.5_37.5'
areadef = get_area_def(area_id, description, proj_id, projection,x_size, y_size, area_extent)

In [9]:
local_scene = scn.resample(areadef)

In [10]:
local_scene.show(rgbname)

/home/a001673/.local/lib/python2.7/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in greater
  return func(*args2)
/home/a001673/.local/lib/python2.7/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in log10
  return func(*args2)


<img src='https://media.githubusercontent.com/media/pytroll/pytroll-examples/321c1cc5799832b8cc5b9e0994a0c461cf11213d/satpy/himawari_ahi_truecolor_201502070300_japan_thumb.png'>